# Mounting google drive
Location where you can view your files saved on drive.
/content/drive/My Drive

PS: to run bash command, ''!" before the command and run the cell.
For eg. "!ls" 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/'My Drive'

In [ ]:
cd /content/drive/My Drive/SarData10Class_Lee

# Imports

To support both Python 2 and Python 3:

In [ ]:
from __future__ import division, print_function, unicode_literals

To plot figures:

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import time

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


# Reproducibility

Reset the default graph (re-run notebook without restarting the kernel):

In [ ]:
tf.reset_default_graph()
#tf.compat.v1.get_default_graph

Set random seed so notebook always produces the same output:

In [ ]:
np.random.seed(42)
tf.set_random_seed(42)

# Load MSTAR

In [ ]:
import numpy as np
import _pickle as pickle
with open('batches.meta','rb') as f1:
    metaData = pickle.load(f1)   
with open('data_batch_1','rb') as f2:
    trainData = pickle.load(f2)   
with open('test_batch','rb') as f3:
    testData = pickle.load(f3)   
mstar=trainData

Visualizing SAR images:

In [ ]:
dim_x=128
dim_y=128
n_samples = 10

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = mstar['data'][index,:].reshape(dim_x, dim_y)
    plt.imshow(sample_image, cmap="gray")
    plt.axis("off")

plt.show()

Test Images

In [ ]:
n_samples = 10
#decoder_output = deconv
sample_images = testData['data'][:n_samples].reshape([-1, 128, 128, 1])

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    sample_image = testData['data'][index,:].reshape(dim_x, dim_y)
    plt.imshow(sample_image, cmap="gray")
    plt.axis("off")



The corresponding labels:

# Input Images

Creating a placeholder for the input images (128×128 pixels, 1 color channel = grayscale).

In [ ]:
X = tf.placeholder(shape=[None, dim_x, dim_y, 1], dtype=tf.float32, name="X")

# Primary Capsules

The first layer will be composed of N maps of, where each capsule will output 
an MD activation vector, here N and M are modified for analyzing time required to train each type of model.

In [ ]:
caps1_n_maps = 1 # so the number of maps will be only 1 (with 256 dim) 
caps1_n_dims = 256 # dimensions of capsule vector'''

'''caps1_n_maps = 2 
caps1_n_dims = 128 # dimensions of capsule vector'''

'''caps1_n_maps = 4 
caps1_n_dims = 64 # dimensions of capsule vector'''

'''caps1_n_maps = 8 
caps1_n_dims = 32 # dimensions of capsule vector '''

'''caps1_n_maps = 16 
caps1_n_dims = 16 # dimensions of capsule vector'''

'''caps1_n_maps = 32 
caps1_n_dims = 8 # dimensions of capsule vector '''

To compute their outputs, we first apply two regular convolutional layers:

In [ ]:
conv1_params = {
    "filters": 256,
    "kernel_size": 9,
    #"kernel_size": 5,
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": 9,
    #"kernel_size": 5,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}

In [ ]:
conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)

In [ ]:
shape = conv2.get_shape().as_list()
caps1_n_caps = caps1_n_maps * int(shape[1] * shape[2])

In [ ]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")
caps1_raw

Defining the `squash()` function, based on equation (1) from the paper:

$\operatorname{squash}(\mathbf{s}) = \dfrac{\|\mathbf{s}\|^2}{1 + \|\mathbf{s}\|^2} \dfrac{\mathbf{s}}{\|\mathbf{s}\|}$

The `squash()` function will squash all vectors in the given array, along the given axis (by default, the last axis).

In [ ]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

Now let's apply this function to get the output $\mathbf{u}_i$ of each primary capsules $i$ :

In [ ]:
caps1_output = squash(caps1_raw, name="caps1_output")
caps1_output

# SAR Capsules

To compute the output of the SAR capsules, we must first compute the predicted output vectors (one for each primary / SAR capsule pair). Then we can run the routing by agreement algorithm.

## Compute the Predicted Output Vectors

The SAR capsule layer contains 10 capsules (one for each class) of 16 dimensions each:

In [ ]:
caps2_n_caps = 10
caps2_n_dims = 16

In [ ]:
init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

Now we can create the first array by repeating `W` once per instance:

In [ ]:
batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")
print(batch_size)

In [ ]:
caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

In [ ]:
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")

In [ ]:
caps2_predicted

For each instance in the batch (batch size unknown yet, therefore size is "?") and for each pair of first and second layer capsules (1152×10) we have created a 16D predicted output column vector (16×1).

## Routing by agreement

Initializing the raw routing weights $b_{i,j}$ to zero:

In [ ]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")

### Round 1

Applying the softmax function to compute the routing weights, $\mathbf{c}_{i} = \operatorname{softmax}(\mathbf{b}_i)$ (equation (3) in the paper):

In [ ]:
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")

Computing the weighted sum of all the predicted output vectors for each second-layer capsule, $\mathbf{s}_j = \sum\limits_{i}{c_{i,j}\hat{\mathbf{u}}_{j|i}}$ (equation (2)-left in the paper):

In [ ]:
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")

Applying the squash function to get the outputs of the second layer capsules at the end of the first iteration of the routing by agreement algorithm, $\mathbf{v}_j = \operatorname{squash}(\mathbf{s}_j)$ :

In [ ]:
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")

In [ ]:
caps2_output_round_1

### Round 2

In [ ]:
caps2_predicted

Look at the shape of `caps2_output_round_1`, which holds 10 outputs vectors of 16D each, for each instance:

In [ ]:
caps2_output_round_1

To match shapes we tile the `caps2_output_round_1` array 1152 times (once per primary capsule) along the second dimension:

In [ ]:
caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

In [ ]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

Updating the raw routing weights $b_{i,j}$ by adding the scalar product $\hat{\mathbf{u}}_{j|i} \cdot \mathbf{v}_j$ we just computed: $b_{i,j} \gets b_{i,j} + \hat{\mathbf{u}}_{j|i} \cdot \mathbf{v}_j$ (Procedure 1, step 7, in the paper).

In [ ]:
raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

In [ ]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")

In [ ]:
caps2_output = caps2_output_round_2
caps2_output


# Estimated Class Probabilities (Length)

In [ ]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")
y_proba

To predict the class of each instance, we select the one with the highest estimated probability.

In [ ]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")
y_proba_argmax
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
y_pred

# Labels

Placeholder for the labels:

In [ ]:
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

# Margin loss

In [ ]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

In [ ]:
T = tf.one_hot(y, depth=caps2_n_caps, name="T")

In [ ]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

In [ ]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 10),
                           name="present_error")

In [ ]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 10),
                          name="absent_error")

In [ ]:
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

Summing the SAR losses for each instance ($L_0 + L_1 + \cdots + L_9$), and computing the mean over all instances. This is the final margin loss:

In [ ]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")

# Reconstruction

Now let's add a decoder network on top of the capsule network. A typical Capsnet achitecture has a regular 3-layer fully connected neural network which will learn to reconstruct the input images based on the output of the capsule network. This will force the capsule network to preserve all the information required to reconstruct the images, across the whole network. This constraint regularizes the model: it reduces the risk of overfitting the training set, and it helps generalize to new images. We have modified this decoder network and designed a transposed convolution network which is computationally less expensive.

## Mask

The paper mentions that during training, instead of sending all the outputs of the capsule network to the decoder network, we must send only the output vector of the capsule that corresponds to the target class. All the other output vectors must be masked out. At inference time, we must mask all output vectors except for the longest one, i.e., the one that corresponds to the predicted class.

In [ ]:
mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")

In [ ]:
reconstruction_targets = tf.cond(mask_with_labels, # condition
                                 lambda: y,        # if True
                                 lambda: y_pred,   # if False
                                 name="reconstruction_targets")

We have the reconstruction targets, so we create the reconstruction mask. It should be equal to 1.0 for the target class, and 0.0 for the other classes, for each instance.

In [ ]:
reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

Let's check the shape of `reconstruction_mask`:

In [ ]:
reconstruction_mask

In [ ]:
reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")

In [ ]:
caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

In [ ]:
caps2_output_masked

In [ ]:
'''decoder_input = tf.reshape(caps2_output_masked,
                           [-1, caps2_n_caps * caps2_n_dims],
                           name="decoder_input")'''

## Decoder

Now let's build a modified decoder (which is different from fully connected network). It has five layers of transposed convolution (also called deconvolution) which is much less expensive than the vanilla fully connected decoder.

In [ ]:
decoder_input = caps2_output_masked
decoder_input = tf.reshape(decoder_input, [-1, 1, 1, 160])

In [ ]:
filters1 = tf.get_variable('w10', [8, 8, 16, 160], initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), dtype=tf.float32)
strides1 = [1, 2, 2, 1]

filters2 = tf.get_variable('w11', [3, 3, 32, 16], initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), dtype=tf.float32)
strides2 = [1, 2, 2, 1]

filters3 = tf.get_variable('w12', [3, 3, 32, 32], initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), dtype=tf.float32)
strides3 = [1, 2, 2, 1]

filters4 = tf.get_variable('w13', [3, 3, 32, 32], initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), dtype=tf.float32)
strides4 = [1, 2, 2, 1]

filters5 = tf.get_variable('w14', [3, 3, 1, 32], initializer=tf.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), dtype=tf.float32)
strides5 = [1, 2, 2, 1]


In [ ]:
output_shape1 = [8, 8, 8, 16]

output_shape2 = [8, 16, 16, 32]

output_shape3 = [8, 32, 32, 32]

output_shape4 = [8, 64, 64, 32]

output_shape5 = [8, 128, 128, 1]



In [ ]:
deconv1 = tf.nn.conv2d_transpose(decoder_input, filters1, output_shape1, strides1, padding='VALID', data_format='NHWC', dilations=None, name=None)
deconv2 = tf.nn.conv2d_transpose(deconv1, filters2, output_shape2, strides2, padding='SAME', data_format='NHWC', dilations=None, name=None)
deconv3 = tf.nn.conv2d_transpose(deconv2, filters3, output_shape3, strides3, padding='SAME', data_format='NHWC', dilations=None, name=None)
deconv4 = tf.nn.conv2d_transpose(deconv3, filters4, output_shape4, strides4, padding='SAME', data_format='NHWC', dilations=None, name=None)
deconv5 = tf.nn.conv2d_transpose(deconv4, filters5, output_shape5, strides5, padding='SAME', data_format='NHWC', dilations=None, name=None)


## Reconstruction Loss

Computing the reconstruction loss. It is the squared difference between the input image and the reconstructed image:

In [ ]:
X_flat = X

squared_difference = tf.square(deconv5 - X_flat,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                    name="reconstruction_loss")

## Final Loss

The final loss is the sum of the margin loss and the reconstruction loss (scaled down by a factor of 0.0005 to ensure the margin loss dominates training):

In [ ]:
alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")

## Accuracy

To measure our model's accuracy, we need to count the number of instances that are properly classified. For this, we can simply compare `y` and `y_pred`, convert the boolean value to a float32 (0.0 for False, 1.0 for True), and compute the mean over all the instances:

In [ ]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

## Training Operations

The paper mentions that the authors used the Adam optimizer with TensorFlow's default parameters:

In [ ]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# Training

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split( trainData['data'], trainData['labels'], test_size=0.20, random_state=42)

In [ ]:
cd /content/drive/'My Drive'

In [ ]:
n_epochs = 5
batch_size = 8
restore_checkpoint = True
n_iterations_per_epoch = X_train.shape[0] // batch_size
print(X_train.shape[0])
print(X_validate.shape[0])
n_iterations_validation = X_validate.shape[0] // batch_size
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"

In [ ]:
with tf.Session() as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        #saver.restore(sess, checkpoint_path)
        print("Here 2")
    else:
        init.run()
        print("Here 3")
    
    avg_time = 0
    for epoch in range(n_epochs):
        curr_batch_size1=0;
        curr_batch_size2=0;
        start_time = time.time()
        #print("Here 4")
        for iteration in range(1, n_iterations_per_epoch + 1):
            #print("Here 5")
            X_batch = X_train[batch_size*curr_batch_size1:batch_size*curr_batch_size1 + batch_size] 
            y_batch = y_train[batch_size*curr_batch_size1:batch_size*curr_batch_size1 + batch_size]

            #print("Heere 6")
            # Run the training operation and measure the loss:
            _, loss_train = sess.run(
                    [training_op, loss],
                    feed_dict={X: X_batch.reshape([-1, dim_x, dim_y, 1]),
                               y: y_batch,
                               mask_with_labels: True})
            

            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
            iteration, n_iterations_per_epoch,
            iteration * 100 / n_iterations_per_epoch,
            loss_train),
            end="")
            curr_batch_size1 += 1

        total_time = time.time() - start_time
        print(f'time for epoch {epoch} is {total_time}')
        avg_time += total_time
        
        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []

        for iteration in range(1, n_iterations_validation + 1):
            X_batch = X_validate[batch_size*curr_batch_size2:batch_size*curr_batch_size2 + batch_size]
            y_batch = y_validate[batch_size*curr_batch_size2:batch_size*curr_batch_size2 + batch_size]
            loss_val, acc_val = sess.run(
                    [loss, accuracy],
                    feed_dict={X: X_batch.reshape([-1, dim_x, dim_y, 1]),
                               y: y_batch})
            
            print(type(loss_val))
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            print("++++++++++ Validation +++++++++++++++++++++++")
            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                      end=" " * 10)
            curr_batch_size2 += 1
            
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val

In [ ]:
avg_time /= n_epochs
print(avg_time)

In [ ]:
batch_size = 8
print(testData['data'].shape[0])

n_iterations_test = testData['data'].shape[0] // batch_size
curr_batch_size2 = 0

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)

    loss_tests = []
    acc_tests = []
    start_time = time.time()
    for iteration in range(1, n_iterations_test + 1):
        
        X_batch = testData['data'][batch_size*curr_batch_size2:batch_size*curr_batch_size2 + batch_size]
        y_batch = testData['labels'][batch_size*curr_batch_size2:batch_size*curr_batch_size2 + batch_size]
        loss_test, acc_test = sess.run(
                [loss, accuracy],
                feed_dict={X: X_batch.reshape([-1, dim_x, dim_y, 1]),
                           y: y_batch})
        loss_tests.append(loss_test)
        acc_tests.append(acc_test)
        print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                  iteration, n_iterations_test,
                  iteration * 100 / n_iterations_test),
              end=" " * 10)
        curr_batch_size2 += 1
    total_time = time.time() - start_time
    loss_test = np.mean(loss_tests)
    acc_test = np.mean(acc_tests)
    print("\rFinal test accuracy: {:.4f}%  Loss: {:.6f}".format(
        acc_test * 100, loss_test))

The time taken to train and test the images is recorded and compared for each type of model ensuring that the accuracy is maintained.